## Adeel Assessor file

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import time

#import sys


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as pl

import numpy as np
np.set_printoptions(precision=2, linewidth=120, suppress=True, edgeitems=4)

import pandas as pd
from pandas import ExcelFile
from pandas import ExcelWriter

pd.set_option('display.max_columns',300)
#pd.set_option('display.max_rows', 150)
#pd.set_option('precision', 5)

import nltk
from nltk.util import ngrams
from collections import Counter
from nltk.corpus import stopwords
import string
stopWords = stopwords.words('english') + list(string.punctuation)

from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

In [3]:
#os.getcwd()
#!ls
os.chdir('/Users/zacklarsen/Dropbox/Datasets/')
!ls

Adeel Qureshi - Assessor.csv
Adeel Qureshi - Assessor.xlsx
AssessorCSV.csv
Axiometrics Trends.xlsx
CA 2.txt
CA.txt
CA_3M.txt
Chuckor_Assessor_Layout_20161103.xlsx
Deed_CA.txt.zip
Exploratory-Rental Crawl Sample_for_Austin.ipynb
Exploratory-Rental+Crawl+Sample.xlsx.py
FirstAmListing.txt
Icon?
Int%2FExt Attribute Example (1).xlsx
RCA Data Sample.xls
Real Capital Analytics Sample.xls
Rent Zip Map.xlsx
Rental Crawl Sample.csv
Rental Crawl Sample.xlsx
Tuesday.csv
Tuesday_new.csv
rentcrawl_addresses.csv
~$Rent Zip Map.xlsx


In [4]:
Adeel_Assessor = pd.read_csv('Adeel Qureshi - Assessor.csv',error_bad_lines=False)

//anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4,6,12,13,14,23,24,26,27,33,38,48,50,56,74,75,79,82,83,84,88,116,132,133,135,137,138,139,140,141,142,143,144,145,147,148,158,159,170,171,175,183,184,194,195) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
Adeel_Assessor.head(5)

,FIPS,PropertyID,FormattedAPN,APNSeqNbr,OldAPN,TaxAccountNumber,SitusStdHouseNbr,SitusStdHouseNbr2,SitusStdHouseNbrSuffix,SitusStdDirectionLeft,SitusStdStreet,SitusStdMode,SitusStdDirectionRight,SitusStdUnitType,SitusStdUnitNbr,SitusStdCity,SitusStdState,SitusStdZIP5,SitusStdZIP4,SitusStdCarrierCode,PropertyClassID,StdLandUseCode,StateLandUseCode,CountyLandUseCode,Zoning,SitusCensusTract,MobileHomeInd,TimeshareCode,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeAcres,LotSizeSqFt,LotIrregularInd,OwnerStd1CorpInd,OwnerStd1LastName,OwnerStd1FirstName,OwnerStd1MiddleName,OwnerStd1Suffix,OwnerStd2CorpInd,OwnerStd2LastName,OwnerStd2FirstName,OwnerStd2MiddleName,OwnerStd2Suffix,OwnerStdNAME1FULL,OwnerStdNAME2FULL,OwnerOccupied,OwnerStd1OwnershipRights,OwnerStd1RelationshipType,MailingStdHouseNbr,MailingStdHouseNbr2,MailingStdHouseNbrSuffix,MailingStdDirectionLeft,MailingStdStreet,MailingStdMode,MailingStdDirectionRight,MailingStdUnitType,MailingStdUnitNbr,MailingStdCity,MailingStdState,MailingStdZIP5,MailingStdZIP4,MailingStdCarrierCode,MailingOptOut,AssdTotalValue,AssdLandValue,AssdImprovementValue,MarketTotalValue,MarketValueLand,MarketValueImprovement,TaxAmount,TaxYear,TaxDeliquentYear1,AssdYear,TaxRateCodeArea,SchoolTaxDistrict1Code,SchoolTaxDistrict2Code,SchoolTaxDistrict3Code,HomesteadInd,VeteranInd,DisabledInd,WidowInd,SeniorInd,SchoolCollegeInd,ReligiousInd,WelfareInd,PublicUtilityInd,CemeteryInd,HospitalInd,LibraryInd,BuildingArea,BuildingAreaInd,SumBuildingSqFt,SumLivingAreaSqFt,SumGroundFloorSqFt,SumGrossAreaSqFt,SumAdjAreaSqFt,SumBasementSqFt,SumGarageSqFt,YearBuilt,EffectiveYearBuilt,Bedrooms,TotalRooms,BathTotalCalc,BathFull,BathsParcialNbr,BathFixturesNbr,AirConditioningCode,BasementCode,BuildingClassCode,BuildingConditionCode,ConstructionTypeCode,ElectricityCode,FireplaceCode,FloorTypeCode,Garage,HeatCode,SiteInfluenceCode,GarageParkingNbr,DrivewayCode,PatioCode,PoolCode,PorchCode,BuildingQualityCode,RoofCoverCode,RoofTypeCode,SewerCode,StoriesNbrCode,StyleCode,SumResidentialUnits,SumBuildingsNbr,SumCommercialUnits,WaterCode,LotCode,LotNumber,LandLot,Block,Section,District,LegalUnit,Municipality,SubdivisionName,SubdivisionPhaseNbr,SubdivisionTractNbr,Meridian,AssessorsMapRef1,LegalDescription,CurrentSaleTransactionId,CurrSaleInputDocNbr,CurrSaleInputBook,CurrSaleInputPage,CurrSaleRecordingDate,CurrSaleContractDate,CurrSaleDocumentType,CurrSalesPrice,CurrSaleBuyer1FullName,CurrSaleBuyer2FullName,CurrSaleSeller1FullName,CurrSaleSeller2FullName,ConCurrMtg1InputDocNbr,ConCurrMtg1InputBook,ConCurrMtg1InputPage,ConCurrMtg1RecordingDt,ConCurrMtg1LoanAmt,ConCurrMtg1Lender,ConCurrMtg1PrivateLender,ConCurrMtg1Term,ConCurrMtg1LoanDueDt,ConCurrMtg1AdjOrFix,ConCurrMtg1LoanType,ConCurrMtg1Typefinancing,ConCurrMtg2InputDocNbr,ConCurrMtg2InputBook,ConCurrMtg2InputPage,ConCurrMtg2RecordingDt,ConCurrMtg2LoanAmt,ConCurrMtg2Lender,ConCurrMtg2PrivateLender,ConCurrMtg2Term,ConCurrMtg2LoanDueDt,ConCurrMtg2AdjOrFix,ConCurrMtg2LoanType,ConCurrMtg2Typefinancing,PrevSaleTransactionId,PrevSaleInputDocNbr,PrevSaleInputBook,PrevSaleInputPage,PrevSaleRecordingDate,PrevSaleContractDate,PrevSaleDocumentType,PrevSalesPrice,PrevSaleBuyer1FullName,PrevSaleBuyer2FullName,PrevSaleSeller1FullName,PrevSaleSeller2FullName,PrevMtg1InputDocNbr,PrevMtg1InputBook,PrevMtg1InputPage,PrevMtg1RecordingDt,PrevMtg1LoanAmt,PrevMtg1Lender,PrevMtg1PrivateLender,PrevMtg1Term,PrevMtg1LoanDueDt,PrevMtg1AdjRider,PrevMtg1LoanType,PrevMtg1Typefinancing,PrevMtg2InputDocNbr,PrevMtg2InputBook,PrevMtg2InputPage,PrevMtg2RecordingDt,PrevMtg2LoanAmt,PrevMtg2Lender,PrevMtg2PrivateLender,PrevMtg2Term,PrevMtg2LoanDueDt,PrevMtg2AdjRider,PrevMtg2LoanType,PrevMtg2Typefinancing,FATimestamp,FAUCIDFlag
0,6039,14297393,004-041-009,NaN,NaN,NaN,824,NaN,NaN,NaN,CUTTING,ST,NaN,NaN,NaN,MADERA,CA,93638.0,2558.0,C010,R,1001.0,NaN,RAXX,NaN,602.0,NaN,NaN,NaN,0.0,147,6388.0,NaN,NaN,PINEDA,CIPRIANO,NaN,NaN,NaN,PINEDA,ISRAEL,NaN,NaN,PINEDA CIPRIANO,PINEDA ISRAEL,NaN,NaN,NaN,17829,NaN,NaN,NaN,FAIRFIELD,DR,NaN

In [5]:
# Sort columns by number of missing values
count_nan = len(Adeel_Assessor) - Adeel_Assessor.count()
count_nan.sort_values(inplace=True,ascending=False)
count_nan
countDF = pd.DataFrame(count_nan)
# countDF.to_csv('CountDF.csv')

ConCurrMtg1Term              455303
SumAdjAreaSqFt               455303
SumGroundFloorSqFt           455303
OwnerStd1RelationshipType    455303
MailingStdHouseNbr2          455303
VeteranInd                   455303
SitusStdHouseNbr2            455303
SumLivingAreaSqFt            455303
RoofTypeCode                 455303
HospitalInd                  455303
DrivewayCode                 455303
SitusStdHouseNbrSuffix       455303
ConCurrMtg2Typefinancing     455303
CemeteryInd                  455303
ConCurrMtg2AdjOrFix          455303
PublicUtilityInd             455303
ConCurrMtg2Term              455303
ConCurrMtg2PrivateLender     455303
MailingOptOut                455303
ConCurrMtg1PrivateLender     455303
ElectricityCode              455303
ConCurrMtg1Typefinancing     455303
SeniorInd                    455303
WidowInd                     455303
SumGrossAreaSqFt             455303
SchoolTaxDistrict3Code       455303
PrevMtg2InputBook            455303
PrevMtg2Typefinancing       

In [12]:
address = Adeel_Assessor.loc[:,['MailingStdHouseNbr','MailingStdDirectionLeft',
                               'MailingStdStreet','MailingStdMode',
                               'MailingStdCity','MailingStdState',
                               'MailingStdZIP5']]

address.head(4)

,MailingStdHouseNbr,MailingStdDirectionLeft,MailingStdStreet,MailingStdMode,MailingStdCity,MailingStdState,MailingStdZIP5
0,17829,NaN,FAIRFIELD,DR,MADERA,CA,93638.0
1,13367,NaN,WOOD,ST,MADERA,CA,93638.0
2,816,NaN,CUTTING,ST,MADERA,CA,93638.0
3,828,NaN,CUTTING,ST,MADERA,CA,93638.0


## Process the address to link up to RentCrawl.xlsx file

In [21]:
testcopy = address.head(10)

In [34]:
testcopy.loc[:,'First'] = testcopy.loc[:,['MailingStdHouseNbr',
                                   'MailingStdDirectionLeft',
                                   'MailingStdStreet',
                                   'MailingStdMode']].apply(lambda x: ' '.join(x.fillna('').map(str)), axis=1)


                
testcopy

//anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,MailingStdHouseNbr,MailingStdDirectionLeft,MailingStdStreet,MailingStdMode,MailingStdCity,MailingStdState,MailingStdZIP5,First,Last,Address
0,17829,NaN,FAIRFIELD,DR,MADERA,CA,93638.0,17829 FAIRFIELD DR,CA 93638.0,"17829 FAIRFIELD DR,"
1,13367,NaN,WOOD,ST,MADERA,CA,93638.0,13367 WOOD ST,CA 93638.0,"13367 WOOD ST,"
2,816,NaN,CUTTING,ST,MADERA,CA,93638.0,816 CUTTING ST,CA 93638.0,"816 CUTTING ST,"
3,828,NaN,CUTTING,ST,MADERA,CA,93638.0,828 CUTTING ST,CA 93638.0,"828 CUTTING ST,"
4,832,NaN,CUTTING,ST,MADERA,CA,93638.0,832 CUTTING ST,CA 93638.0,"832 CUTTING ST,"
5,815,NaN,AUSTIN,ST,MADERA,CA,93638.0,815 AUSTIN ST,CA 93638.0,"815 AUSTIN ST,"
6,817,NaN,AUSTIN,ST,MADERA,CA,93638.0,817 AUSTIN ST,CA 93638.0,"817 AUSTIN ST,"
7,831,NaN,AUSTIN,ST,MADERA,CA,93638.0,831 AUSTIN ST,CA 93638.0,"831 AUSTIN ST,"
8,710,E,LINCOLN,AVE,MADERA,CA,93638.0,710 E LINCOLN AVE,CA 93638.0,"710 E LINCOLN AVE,"
9,7644,N,PALM,AVE,FRESNO,CA,93711.0,7644 N PALM AVE,CA 93711.0,"7644 N PALM AVE,"


In [24]:
testcopy['Last'] = testcopy.loc[:,['MailingStdState',
                                   'MailingStdZIP5']].apply(lambda x: ' '.join(x.fillna('').map(str)), axis=1)
 
testcopy

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,MailingStdHouseNbr,MailingStdDirectionLeft,MailingStdStreet,MailingStdMode,MailingStdCity,MailingStdState,MailingStdZIP5,First,Last
0,17829,NaN,FAIRFIELD,DR,MADERA,CA,93638.0,17829 FAIRFIELD DR,CA 93638.0
1,13367,NaN,WOOD,ST,MADERA,CA,93638.0,13367 WOOD ST,CA 93638.0
2,816,NaN,CUTTING,ST,MADERA,CA,93638.0,816 CUTTING ST,CA 93638.0
3,828,NaN,CUTTING,ST,MADERA,CA,93638.0,828 CUTTING ST,CA 93638.0
4,832,NaN,CUTTING,ST,MADERA,CA,93638.0,832 CUTTING ST,CA 93638.0
5,815,NaN,AUSTIN,ST,MADERA,CA,93638.0,815 AUSTIN ST,CA 93638.0
6,817,NaN,AUSTIN,ST,MADERA,CA,93638.0,817 AUSTIN ST,CA 93638.0
7,831,NaN,AUSTIN,ST,MADERA,CA,93638.0,831 AUSTIN ST,CA 93638.0
8,710,E,LINCOLN,AVE,MADERA,CA,93638.0,710 E LINCOLN AVE,CA 93638.0
9,7644,N,PALM,AVE,FRESNO,CA,93711.0,7644 N PALM AVE,CA 93711.0


In [32]:
testcopy.loc[:,['Address']] = testcopy.loc[:,('First','MailingStdCity'
                                   'Last')].apply(lambda x: ','.join(x.fillna('').map(str)), axis=1)
 

//anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [33]:
testcopy

,MailingStdHouseNbr,MailingStdDirectionLeft,MailingStdStreet,MailingStdMode,MailingStdCity,MailingStdState,MailingStdZIP5,First,Last,Address
0,17829,NaN,FAIRFIELD,DR,MADERA,CA,93638.0,17829 FAIRFIELD DR,CA 93638.0,"17829 FAIRFIELD DR,"
1,13367,NaN,WOOD,ST,MADERA,CA,93638.0,13367 WOOD ST,CA 93638.0,"13367 WOOD ST,"
2,816,NaN,CUTTING,ST,MADERA,CA,93638.0,816 CUTTING ST,CA 93638.0,"816 CUTTING ST,"
3,828,NaN,CUTTING,ST,MADERA,CA,93638.0,828 CUTTING ST,CA 93638.0,"828 CUTTING ST,"
4,832,NaN,CUTTING,ST,MADERA,CA,93638.0,832 CUTTING ST,CA 93638.0,"832 CUTTING ST,"
5,815,NaN,AUSTIN,ST,MADERA,CA,93638.0,815 AUSTIN ST,CA 93638.0,"815 AUSTIN ST,"
6,817,NaN,AUSTIN,ST,MADERA,CA,93638.0,817 AUSTIN ST,CA 93638.0,"817 AUSTIN ST,"
7,831,NaN,AUSTIN,ST,MADERA,CA,93638.0,831 AUSTIN ST,CA 93638.0,"831 AUSTIN ST,"
8,710,E,LINCOLN,AVE,MADERA,CA,93638.0,710 E LINCOLN AVE,CA 93638.0,"710 E LINCOLN AVE,"
9,7644,N,PALM,AVE,FRESNO,CA,93711.0,7644 N PALM AVE,CA 93711.0,"7644 N PALM AVE,"


In [12]:
len(Adeel_Assessor['FormattedAPN'])

455303

In [10]:
len(Adeel_Assessor['FormattedAPN'].unique())

447431

In [11]:
len(Adeel_Assessor['PropertyID'].unique())

448530